# BENCHMARKING

In [2]:
import pandas as pd 
import numpy as np
import warnings
import time
import math

import logging
from pythonjsonlogger import jsonlogger

### Setup Logging

In [2]:
import logging
import logging.config
from codes.logging_model import custom_log_record_factory, CustomJsonFormatter
import datetime

today = datetime.datetime.now().strftime("%Y-%m-%d")

# Configure logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Create a file handler
file_handler = logging.FileHandler(f'model_logs{today}.json')
file_handler.setLevel(logging.DEBUG)

# Create and set custom formatter
formatter = CustomJsonFormatter('%(asctime)s %(name)s %(levelname)s %(message)s %(custom_field)s')
# formatter = jsonlogger.JsonFormatter()
file_handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(file_handler)

## Data Processing

In [4]:
## PARAMETER - CASE 0
PARAMS = {"warehouse": "HSC"
          ,"spec_name": "JSH270C-PO" # yeu cau chuan hoa du lieu OP - PO
          ,"thickness": 1.6
          ,"maker" : "CSVC"
          ,"stock_ratio": { #get from app 
                    "limited": None
                    # "default": 2
                    # "user_setting": 4
                }
        #   ,"forecast_scenario": median
          }

margin_dict = { "HSC":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_2.6": {
        "thickness": 2.6,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2,
        "margin": 8
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    },
    "NQS":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_2.6": {
        "thickness": 2.6,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2,
        "margin": 8
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    }
}

# DOMAIN KNOWLEDGE:
# - User usually overcut current order_quantity (30 -50%) default on model
# - Users want to cut all PO qty in one stock cut ( increase boundary to max 3 months forecast)

# GET ALL PARAMS
MIN_MARGIN = margin_dict[PARAMS["warehouse"]][f"thickness_{PARAMS["thickness"]}"]["margin"]
print(f"MIN_MARGIN:{MIN_MARGIN}")

BOUND_KEY = next(iter(PARAMS['stock_ratio']))
BOUND_VALUE = PARAMS['stock_ratio'][BOUND_KEY]
print(f"BOUND_KEY:{BOUND_KEY}, BOUND_VALUE:{BOUND_VALUE}")
LIMIT_BOUND_KEY, LIMIT_BOUND_VALUE =  "limited", None

MIN_MARGIN:6
BOUND_KEY:limited, BOUND_VALUE:None


In [4]:
logger.info('Model loading', extra={'model_params': PARAMS})

INFO:__main__:Model loading


In [5]:
from typing import Dict, Any

def check_finish_weight_per_stock(weight_s: float, width_s: float, finish: Dict[str, Dict[str, Any]], BOUND_KEY: str) -> Dict[str, bool]:
    """
    DOMAIN KNOWLEDGE:
    - As upper_demand is slightly/or significant higher than need_cut, 
    - We can check to pick stock that have weight per slice cut < upper_demand
    -  So that, at least, we can cut 1 slice of the FG with that kind of stock

    PICK STOCK WITH ALL/ALMOST TRUE VALUE (SUM TRUE LARGEST)
    """
    wu = weight_s / width_s # MC WEIGHT PER UNIT
    weight_f = {f: finish[f]["width"]*wu for f in finish.keys()} # dict of weight FG on the chosen stock
    f_upper_demand = {f: finish[f][f"upper_bound_{BOUND_KEY}"] for f in finish.keys()} # dict of upperbound demand FG on the chosen stock

    check_f ={f: weight_f[f] < f_upper_demand[f] for f in finish.keys()} # dict of Yes/No 

    return check_f

In [6]:
from codes.process_df import filter_by_params
file_path = "data/test_mc_df.xlsx"
stock_key, st_value_columns = "inventory_id", ['receiving_date','status',"width", "weight"]
stock_df = filter_by_params(file_path, PARAMS)

file_path = "data/test_finish_df_Jul2022.xlsx"
finish_key, fn_value_columns = "order_id", ["width", "need_cut", f"upper_bound_{BOUND_KEY}", "fc1", "fc2", "fc3"]
finish_df = filter_by_params(file_path,PARAMS)

#### DOMAIN KNOWLEDGE:

Chon stock trim loss thap, overcut thap, weight_loss

=> 0 loc finish, stock df, stop khi ko co need cut <0 , be hon lower bound
+ 1 dkien de vao lai pool optimize
- 1.1 ko co need cut lon
- 1.2 ko co stock de cat
+  2 dem di optimize
- 2.1 add them finish de thu optimize
- 2.2 tang muc cat du (detect o phan weight constraint - neu constraint << cat 1 dai, width qua nho)
=> 0 update lai vao finish, stock df

In [7]:
from codes.process_df import filter_stock_df_to_dict, filter_finish_df_to_dict, check_need_cut_qty

finish = filter_finish_df_to_dict(finish_df, finish_key, fn_value_columns, BOUND_VALUE)
sum_of_total_need_cuts = check_need_cut_qty(finish_df)
print(f"No.of PO fin: {len(finish)}, Need-cut qty: {sum_of_total_need_cuts}")
      
# =================================================================
# CASE 1:
# TAKE NORMAL STOCK - NOT REWIND/ SEMI
# - DOMAIN KNOWLEDGE:
# # pick ~ stock co weight > need cut am, so luong stock ~= finish
stock_df = stock_df[~stock_df['status'].isin(['R:REWIND'
                            ,'Z:SEMI MCOIL'
                            ,'S:SEMI FINISHED'])]

stocks = filter_stock_df_to_dict(stock_df, stock_key, st_value_columns, sum_of_total_need_cuts)
print(f"No.of available stocks: {len(stocks)}")

# ================================================================
# # CASE 2
# # TAKE REWIND, SEMI TO CUT FIRST 
# stock_df = stock_df[stock_df['status'].isin(['R:REWIND'
#                             ,'Z:SEMI MCOIL'
#                             ,'S:SEMI FINISHED'])]

# stocks = filter_stock_df_to_dict(stock_df, stock_key, st_value_columns, sum_of_total_need_cuts)
# print("No.of available stock: ")
# print(len(stocks))

No.of PO fin: 4, Need cut qty: 10100
No.of available stock: 2


In [8]:
for s in stocks.keys():
    check_f = check_finish_weight_per_stock(stocks[s]["weight"],stocks[s]["width"], finish, BOUND_KEY)
    print(check_f)

{'F13': True, 'F12': True, 'F11': True, 'F14': True}
{'F13': True, 'F12': True, 'F11': True, 'F14': True}


In [ ]:
logger.info('Check stock and finish', extra={'temp_results': check_f})

## Optima - Base Case AMPL

In [10]:
from codes.create_patterns import *
naive_patterns = make_patterns_by_weight_width(stocks, finish, BOUND_KEY, MIN_MARGIN)
# display(naive_patterns)

In [ ]:
from codes.optima_sol import solve_cutting_stock_ampl

for s in stocks.keys():
    opt_patterns = solve_cutting_stock_ampl(stocks[s]["width"],stocks[s]["weight"],finish, naive_patterns,MIN_MARGIN,BOUND_KEY)
    if len(opt_patterns) == 0:
        print(f"No optimal solution with stock {s}")
        print("=================================")
    else:
        print(f"Solution for stock {s}")
        print(opt_patterns)
        trim_loss = stocks[s]["width"] - sum([finish[f]["width"]*opt_patterns[f] for f in finish.keys()])
        weight_loss = trim_loss * stocks[s]["weight"]/stocks[s]["width"]
        print(f"stock width: {stocks[s]["width"]},trim loss :{trim_loss}, weight loss: {weight_loss}")

In [ ]:
# from codes.errors_handling import run_with_timeout
SOLVER_MILO = "highs"
SOLVER_MINLO = "ipopt"

from amplpy import AMPL, ampl_notebook
from codes.optima_sol import cut_stock_by_patterns

ampl = ampl_notebook(
    modules=["coin", "highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

for s in stocks.keys():
    # create pattern by stocks
    patterns = []
    min_cuts_dict, max_cuts_dict  = ap_stock_bound(naive_patterns,finish,s)
    print(f"Start create combination for stock {s}")
    generate_cut_combinations_with_timeout(s, min_cuts_dict, max_cuts_dict, patterns, TIMEOUT = 40)
    # patterns = run_with_timeout(generate_cut_combinations, args=(s, min_cuts_dict, max_cuts_dict, patterns), timeout=timeout_limit)
    if patterns is None:
        print("Null value")
    else:
        opt_patterns = cut_stock_by_patterns(
                                     stocks[s]["width"]
                                     ,stocks[s]["weight"] 
                                     ,finish 
                                     ,patterns
                                     ,BOUND_KEY
                                     ,MIN_MARGIN
                                     )

        if len(opt_patterns) == 0:
            print(f"No optimal solution with stock {s}")
            print("=================================")
        else:
            print(f"Solution for stock {s}")
            for p in opt_patterns:
                print(f"take pattern {p}")
                print("*****")
                cuts_dict= patterns[p]['cuts']
                print(cuts_dict)
                trim_loss = stocks[s]["width"] - sum([finish[f]["width"]*cuts_dict[f] for f in finish.keys()])
                weight_loss = trim_loss * stocks[s]["weight"]/stocks[s]["width"]
                print(f"trim loss :{trim_loss}, weight loss: {weight_loss}")
            print("=================================")

## Optima - Base Case ORTOOLS

In [13]:
from ortools.linear_solver import pywraplp
from codes.optima_sol import solve_cutting_stock_ortools

solution = []
for s in stocks.keys():
    print(f" Use stock id {s}")
    solve_cutting_stock_ortools(finish,stocks[s]["width"], stocks[s]["weight"], MIN_MARGIN, BOUND_KEY, naive_patterns)
    # solution.append({f'stock_id: {s}, trim_loss: {trim_loss},weight_loss: {weight_loss}'})

 Use stock id S487
Solution:
[F13] = 1.0
[F12] = 3.0
[F11] = 1.0
[F14] = 2.0
 Use stock id S496
Solution:
[F13] = 1.0
[F12] = 3.0
[F11] = 1.0
[F14] = 2.0


## Optima - Base Case CXVPY

In [ ]:
import cvxpy as cp

def cut_patterns_cv(stocks, finish, patterns):
    # Define sets
    F = list(finish.keys())
    P = list(range(len(patterns)))

    # Parameters
    c = [stocks[patterns[p]["stock"]]["cost"] for p in P]
    a = {(f, p): patterns[p]["cuts"].get(f, 0) for p in P for f in F}
    demand_finish = {f: finish[f]["demand"] for f in F}

    # Variables
    x = cp.Variable(len(P), integer=True, nonneg=True)

    # Objective function: minimize cost
    cost = cp.sum([c[p] * x[p] for p in P])

    # Constraints
    constraints = []
    for f in F:
        constraints.append(cp.sum([a[f, p] * x[p] for p in P]) >= demand_finish[f])

    # Formulate the problem
    problem = cp.Problem(cp.Minimize(cost), constraints)

    # Solve the problem
    problem.solve(solver=cp.CPLEX)

    return x.value, problem.value

# Example usage:
x, cost = cut_patterns_cv(stocks, finish, patterns)
print(f"Optimal pattern choice: {x}")
print(f"Minimum cost: {cost}")

## Optima - Base Case PuPL